In [6]:
import pyconll
import pyconll.util
import re
import nltk
import wikipedia
import collections

In [7]:
# Read Page infomation from wikipedia dataset
title = ['Barack Obama','Donald Trump','Kobe Bryant']
dataset = [wikipedia.page(t) for t in title]

In [55]:
# Extract all the sentences that contains link
pages = collections.defaultdict(dict)
for d in dataset:
    print('Preprocessing ', d.title)
    for i,link in enumerate(d.links):
        if i % 100 == 0:
            print('Finishing {0}th link'.format(i))
        if link in d.content:
            index = d.content.index(link)
            start_index = index
            end_index = index + len(link)
            while start_index > 0:
                if d.content[start_index] in '.!?=\n\t':
                    
                    tmpIndex = start_index
                    while d.content[tmpIndex] != ' ':
                        tmpIndex -= 1
                    found = False
                    for l in d.links:
                        if d.content[tmpIndex+1:start_index+1] in l:
                            found = True
                            break
                    if found:
                        start_index -= 1
                        continue
                        
                    start_index += 1
                    break
                start_index -= 1
            while end_index < len(d.content):
                if d.content[end_index] in '.!?=\n\t':
                    
                    tmpIndex = end_index
                    while d.content[tmpIndex] != ' ':
                        tmpIndex -= 1
                    found = False
                    for l in d.links:
                        if d.content[tmpIndex+1:end_index+1] in l:
                            found = True
                            break;
                    if found:
                        end_index += 1
                        continue
                        
                    break
                end_index += 1
            sentence = d.content[start_index:end_index+1]
            pages[d.title][link] = sentence

Preprocessing  Barack Obama
Finishing 0th link
Finishing 100th link
Finishing 200th link
Finishing 300th link
Finishing 400th link
Finishing 500th link
Finishing 600th link
Finishing 700th link
Finishing 800th link
Finishing 900th link
Finishing 1000th link
Finishing 1100th link
Finishing 1200th link
Finishing 1300th link
Finishing 1400th link
Finishing 1500th link
Finishing 1600th link
Finishing 1700th link
Finishing 1800th link
Finishing 1900th link
Finishing 2000th link
Preprocessing  Donald Trump
Finishing 0th link
Finishing 100th link
Finishing 200th link
Finishing 300th link
Finishing 400th link
Finishing 500th link
Finishing 600th link
Finishing 700th link
Finishing 800th link
Finishing 900th link
Finishing 1000th link
Finishing 1100th link
Finishing 1200th link
Finishing 1300th link
Finishing 1400th link
Finishing 1500th link
Preprocessing  Kobe Bryant
Finishing 0th link
Finishing 100th link
Finishing 200th link
Finishing 300th link
Finishing 400th link
Finishing 500th link
Fin

In [64]:
# Check the number of sentences we need to preprocess
print('Number of Sentences containing links: ',sum(len(sentences[s]) for s in sentences))

Number of Sentences containing links:  923


In [63]:
sentences = collections.defaultdict(list)
# Tokenize each sentence and pos tag them
for p in pages:
    for s in pages[p]:
        sentence = pages[p][s]
        tokens = nltk.word_tokenize(sentence)
        upos = nltk.pos_tag(tokens,tagset='universal')
        xpos = nltk.pos_tag(tokens)
        for i,u in enumerate(upos):
            if u[1] == '.':
                u[1] == 'PUNCT'
        sentences[p].append([upos,xpos])

In [77]:
# Write out the file 
f = open('wiki.conll',"w+")
for s in sentences:
    f.write("# Wikipedia Page for {0}\n".format(s))
    l = sentences[s]
    for pos in l:
        upos = pos[0]
        xpos = pos[1]
        for i,p in enumerate(upos):
            f.write("{0}\t{1}\t_\t{2}\t{3}\t_\t_\t_\t_\t_\n".format(i+1,upos[i][0],upos[i][1],xpos[i][1]))
        f.write('\n')
f.close()